## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [2]:
from sklearn import metrics, datasets
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
dia = datasets.load_diabetes()
dia.keys()

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(dia.data, dia.target, test_size= 0.2, random_state= 42)
GDBS_res= GradientBoostingRegressor()
GDBS_res.fit(X_train, Y_train)
Y_pred= GDBS_res.predict(X_test)
mse = metrics.mean_squared_error(Y_test, Y_pred)
print('MSE:',mse)

MSE: 2901.37120151961


In [8]:
n_estimators = [100, 200, 300, 400, 500, 600]
max_depth = [1, 3, 5, 7, 9, 11]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

grid_search = GridSearchCV(GDBS_res, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1, cv= 5)
grid_result = grid_search.fit(X_train, Y_train)
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.9s


Best Accuracy: -3290.471726 using {'max_depth': 1, 'n_estimators': 200}


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   10.9s finished


In [9]:
grid_result.best_params_

{'max_depth': 1, 'n_estimators': 200}

In [11]:
# 使用最佳參數重新建立模型
res_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
res_bestparam.fit(X_train, Y_train)

# 預測測試集
y_pred = res_bestparam.predict(X_test)

In [12]:
print(metrics.mean_squared_error(Y_test, y_pred))

2774.5910277121893
